In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

In [4]:
import shutil

In [5]:
from ShallowMindBot import *
from TowerDefense import *
from TowerDefenseApi import *

In [6]:
towerDefense = TowerDefense('state.json')

In [7]:
api = TowerDefenseApi(towerDefense.getGameState())

In [8]:
bot = ShallowMindBot()

In [9]:
bot.doTurn(api)

Selected action type 4 from options [0, 1, 2, 4, 6] with probabilities [0.21018274 0.19952657 0.19744639 0.19700019 0.1958441 ]


FileNotFoundError: [Errno 2] No such file or directory: '.\\logs\\game_-1\\round_74.pkl'

In [ ]:
possible_actions = [0,1,2,6]
p, h = bot.policy_forward(bot.computeStateFeatures(api), possible_actions)

In [ ]:
p

In [ ]:
selected_action = 1

In [ ]:
h

In [ ]:
NUM_POSSIBLE_ACTIONS = 7

In [10]:
def softmax_grad(softmax):
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [ ]:
x = np.array([float(val) for val in bot.computeStateFeatures(api).values()])

In [11]:
def gradW2(p, possible_actions, selected_action, h):
    dsoftmax = softmax_grad(p)[selected_action,:]
    dlog = dsoftmax / p[selected_action]
    # Fill in dlog with zeros at the actions that were not possible
    dlog_filled = np.zeros(NUM_ACTION_TYPES)
    dlog_filled[possible_actions] = dlog
    dW2 = dlog_filled[None,:].T.dot(h[None,:])
    return dW2, dlog_filled

In [ ]:
gradW2, dlog_filled = gradW2(p, possible_actions, selected_action, h)

In [12]:
def gradW1(dlog_filled, W2, h, x):
    dh = W2.T.dot(dlog_filled)
    dh[h<=0] = 0
    dW1 = np.outer(dh,x)
    return dW1

In [ ]:
gradw1 = gradW1(dlog_filled, bot.model['W2'], h, x)

## Start training

In [13]:
num_batches = 1
batch_size = 100 # used to perform a RMS prop param update every batch_size steps
learning_rate = 1e-3 # learning rate used in RMS prop
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
reward_win = 10**5
reward_lose = -reward_win

In [ ]:
for batch_idx in range(num_batches):
    # Clear out the logs folder
    log_subfolders = [d for d in os.listdir(os.path.join('.','logs')) if os.path.isdir(os.path.join('.','logs',d))]
    for subfolder in log_subfolders:
        shutil.rmtree(os.path.join('.','logs',subfolder))
        
    # Run the games in the batch
    for game_idx in range(batch_size):
        ! java -jar tower-defence-runner-3.0.3.jar

In [45]:
batch_idx = 0
# Learn from the game outcomes: compute and execute gradient update of parameters
gradW1 = np.zeros_like(bot.model['W1'])
gradW2 = np.zeros_like(bot.model['W2'])
for game_idx in range(batch_size):
    num_rounds = max([int(f.split('.')[0].split('_')[1]) for f in os.listdir(os.path.join('.','logs','game_{}'.format(game_idx)))])
    rewards = np.zeros(num_rounds)
    for round_idx in range(num_rounds):
        with open(os.path.join('.','logs','game_{}'.format(game_idx),'round_{}.pkl'.format(round_idx)), 'rb') as f:
            state_features, possible_action_types, action_type_probabilities, hidden_layer, action_type_choice = pickle.load(f)
        rewards[round_idx] = state_features['myMinusOppScore']
        # If last round, estimate who won or lost and update rewards
        if round_idx == (num_rounds-1):
            if state_features['myMinusOppHealth'] > 0:
                rewards[round_idx] += reward_win
            else:
                rewards[round_idx] += reward_lose
    # Compute advantages
    advantages = 
    # Update gradients

In [50]:
np.cumsum(rewards[::-1]) - rewards[-1]

array([    0.,  2223.,  4394.,  6305.,  8141.,  9953., 11648., 13165.,
       14642., 16095., 17443., 18527., 19601., 20648., 21676., 22472.,
       23320., 24144., 24949., 25575., 26235., 26870., 27482., 27998.,
       28501., 29056., 29582., 29984., 30378., 30754., 31117., 31312.,
       31508., 31670., 31820., 31962., 32104., 32246., 32373., 32499.,
       32608., 32734., 32842., 32941., 33042., 33143., 33234., 33316.,
       33402., 33485., 33575., 33662., 33745., 33815., 33878., 33933.,
       33986., 34033., 34073., 34107., 34132., 34148., 34156., 34165.,
       34168., 34168., 34168., 34168., 34168., 34168., 34168., 34168.,
       34168., 34168.])